# LSTM CAMELS-ES: entrenamiento y validación
***

***Autor:** Jesús Casado Rodríguez*<br>
***Fecha:** 16-09-2023*<br>

**Introducción:**<br>
En este _notebook_ se entrena y valida el modelo LSTM que replica el caudal observado del conjunto de datos CAMELS-ES.

**Por hacer**:

In [1]:
%%capture
! pip install neuralhydrology

In [1]:
import pickle
import pandas as pd
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns
import torch

from neuralhydrology.utils.config import Config
from neuralhydrology.evaluation import metrics
from neuralhydrology.nh_run import start_run, eval_run

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/TFM/notebook/models/
from model_utils import *
%cd ./EFAS5/

/content/drive/MyDrive/TFM/notebook/models
/content/drive/MyDrive/TFM/notebook/models/EFAS5


### Configuración

In [3]:
# cargar archivo de configuración
config_file = Path('config_end.yml')
cfg = Config(config_file)

target = cfg.target_variables[0]

print(cfg.experiment_name)
print('epochs:', cfg.epochs, sep='\t\t')
print('hidden size:', cfg.hidden_size, sep='\t')
print('batch size:', cfg.batch_size, sep='\t')
print('dropout:', cfg.output_dropout, sep='\t')
print('clip gradients:', cfg.clip_gradient_norm, sep='\t')

CAMELS-ES_end
epochs:		45
hidden size:	128
batch size:	256
dropout:	0.5
clip gradients:	0.8


### Entrenamiento

In [29]:
# by default we assume that you have at least one CUDA-capable NVIDIA GPU
if torch.cuda.is_available():
    start_run(config_file=config_file)

# fall back to CPU-only mode
else:
    start_run(config_file=config_file, gpu=-1)

# Validation: 100%|██████████| 100/100 [00:19<00:00,  5.11it/s]


### Simulación

In [4]:
run_dir = Path(f'runs/{cfg.experiment_name}_1609_131613')

In [31]:
# periodo de entrenamiento
for epoch in np.arange(1, cfg.epochs + 1):
    eval_run(run_dir=run_dir, period='train', epoch=epoch, gpu=0)
# eval_run(run_dir=run_dir, period='train', epoch=cfg.epochs, gpu=0)

# Evaluation: 100%|██████████| 163/163 [01:31<00:00,  1.79it/s]


In [ ]:
# periodo de validación
#eval_run(run_dir=run_dir, period='validation', gpu=0)

In [32]:
# periodo de evaluación
for epoch in np.arange(1, cfg.epochs + 1):
    eval_run(run_dir=run_dir, period='test', epoch=epoch, gpu=0)
# eval_run(run_dir=run_dir, period='test', epoch=cfg.epochs, gpu=0)

# Evaluation: 100%|██████████| 53/53 [00:38<00:00,  1.37it/s]
